## Example 🖼️
| Thumbnail | Video (*± Middle Frame*) |
|:---:|:---:|
| <img src="https://raw.githubusercontent.com/kubinka0505/YTMVG/master/Documents/Pictures/Examples/Google_Colab/Thumbnail.jpg" height=200> | <img src= "https://raw.githubusercontent.com/kubinka0505/YTMVG/master/Documents/Pictures/Examples/Google_Colab/Cover.jpg" height=200> |
| *Subjectively basing on the image clusters, a<br>16:9 background is generated, onto the<br>center of which the image is placed.* | *Generates the video with dimensions that<br>match the input image, and appends<br>the specified audio to it.*|

---
<br>

### 📱 **Information for mobile users**
<font color="red"><b>If cells don't output anything and are finished immediately, please re-run the site in the <code>Desktop site</code> mode</b></font> (Chrome)

<br>

---
<br>

[![](https://img.shields.io/badge/github-open-lightgray?logo=github&logoColor=white&style=for-the-badge)](https://github.com/kubinka0505/YTMVG)

In [ ]:
#@title # Download Requirements 📥
#@markdown ℹ️ Runtime will be automatically restarted
#@markdown after installation - don't panic.

import os

#-=-=-=-#

os.chdir("/content")
!rm -rf YTMVG
!git clone https://github.com/kubinka0505/YTMVG

os.chdir("YTMVG")
!apt-get install -qq -o=Dpkg::Use-Pty=0 jpegoptim
print()
%pip install -r requirements.txt | grep -v "already satisfied"

#-=-=-=-#

os.kill(os.getpid(), 9)

# Processing ⚙️

In [ ]:
#@title # <font color=lime>**0**</font>. Setup code ⚙️

from os import *
from sty import fg
from PIL import Image
from time import time
from pathlib import Path
from shutil import rmtree
from zipfile import ZipFile
from requests import Session
from google.colab import files
from mutagen import File as mFile
from PIL.ImageColor import getrgb as RGB
del open

try:
	chdir("/content")
except NameError:
	raise SystemExit("Requirements were not installed!\nPlease run the previous cell.")

#-=-=-=-#

__name__	= "YTMVG"
__author__	= "kubinka0505"
__credits__	= __author__
__version__	= "1.1"
__date__	= "01.01.2023"

#-=-=-=-#

session = Session()
session.headers.update(
	{
		"User-Agent":
		"Mozilla/5.0 (Windows NT 10.0; Win32; x32) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
	}
)
get = session.get

class Styles:
	"""Colored Prints."""
	OK = "#0C0"
	Info = "#4AF"
	Flaw = "#F25"
	Error = "#C10"
	Warning = "#FC0"

Class = Styles
for Variable in list(vars(Class))[2:-2]:
	exec('{0}.{1} = "{2}"'.format(
		Class.__name__, Variable,
		fg(*RGB(getattr(Class, Variable)))
		)
	)

Styles.Reset = fg.rs

#-=-=-=-#

def Error(Traceback: str) -> SystemExit:
	"""Custom `SystemExit` handler."""
	raise SystemExit(Styles.Error + Traceback + Styles.Reset + "\a")

__SepSTR = "─" * 32

File_Dir = "/content/YTMVG_Batch"
__Output = path.join(File_Dir, "Processed")

In [ ]:
#@title # <font color=lime>**1.**</font> Upload Files 📤
#@markdown ---
#@markdown #@a
#@markdown ℹ️ Formats:
#@markdown - Audio files:&emsp;&emsp;&ensp;`MP3`/`FLAC`/`OGG`
#@markdown - Cover file:&emsp;&emsp;&emsp;`PNG`/`JPG`/`JPEG`/`WebP`/`BMP`
#@markdown ---

#@markdown ```markdown
#@markdown # You will be asked to upload a text file containing URLs
#@markdown # separated by a newline for each audio and cover file,
#@markdown # where the cover file is separated by two.
#@markdown
#@markdown ``` FILES ARE NOT VERIFIED !!! ```
#@markdown - It should look like this:
#@markdown
#@markdown https://example.com/Audio.mp3
#@markdown https://website.org/Recording.ogg
#@markdown https://web.net/Song.flac
#@markdown
#@markdown https://example.com/Image.png
#@markdown ```
URL_Links_File = True #@param {type: "boolean"}
#@markdown ---
#@markdown #@a
#@markdown ⚠️ **Warning**

#@markdown <font color=red>**In case of manual interruption, <u>all
#@markdown possibly previously uploaded files are ignored!**</u></font>

#-=-=-=-#

try:
	chdir("/content")
except NameError:
	raise SystemExit('Code was not set up!\nPlease run the "Setup/Setup code" cell.')

try:
	mkdir(File_Dir)
except FileExistsError:
	rmtree(File_Dir)
	mkdir(File_Dir)
finally:
	chdir(File_Dir)

#-=-=-=-#
# URL LINKS FILE

__AMU = 0
if URL_Links_File:
	print("Upload {0}TEXT{2} file.\t{1}This can be cancelled.{2}".format(
		Styles.OK, Styles.Info, Styles.Reset)
	)
	__AMU = 1
	try:
		try:
			File = list(files.upload().keys())[0]
		except IndexError:
			raise KeyboardInterrupt
		URLs = open(File, encoding = "U8").readlines()
		URLs = [Line.strip("\n") for Line in URLs if Line != ""]
		URLs = list(set(URLs))[1:]
		remove(File)
		#-=-=-=-#
		for URL in URLs:
			URL = URL.split("?")[0].split("#")[0].split("&")[0]
			FileName = URL.split("/")[-1]
			#-=-=-=-#
			Site = get(URL, stream = 1)
			if Site.ok:
				with open(FileName, "wb") as File_:
					File_.write(Site.content)
			else:
				Error('"{0}" returned status code {1} - {2}, exiting.'.format(
					FileName, Site.status_code, Site.reason
					)
				)
		#-=-=-=-#
		print("{1}Successfully saved {0} files!{2}".format(
			len(URLs), Styles.OK, Styles.Reset
			)
		)
		print("{0}<WARNING>{2} | {1}Aborting manual upload.{2}".format(
			Styles.Info, Styles.Warning, Styles.Reset
			)
		)
	except KeyboardInterrupt:
		__AMU = 0
		print("{0}<WARNING>{2} | {1}URL Links File upload aborted, defaulting to manual.{2}".format(
			Styles.Info, Styles.Warning, Styles.Reset
			)
		)
		print("\n" + __SepSTR + "\n")

#-=-=-=-#
# MANUAL UPLOAD

Type = "Audio", "Cover"
try:
	# Aborting manual upload
	if all((URL_Links_File, __AMU)):
		raise KeyboardInterrupt
	#---#
	for FileType in Type[::-1]:
		print("Upload {2}{0}{3} file{1}...".format(
			FileType.upper(),
			"s" if FileType == Type[0] else "",
			Styles.Info, Styles.Reset
			)
		)
		Files = files.upload().keys()
		if Files:
			for File in Files:
				FileName = File.split(sep)[-1]
				# Verifying
				print('{1}Verifying "{0}"...{2}'.format(
					FileName,
					Styles.OK, Styles.Flaw
					)
				)
				try:
					if FileType == Type[0]:
						_ = mFile(File).info.length
					else:
						_ = Image.open(File)
				except (AttributeError, OSError):
					Error(f'{FileType} file is broken. ("{FileName}")')
				#---#
			print(Styles.Reset)
		else:
			raise KeyboardInterrupt
except KeyboardInterrupt:
	Error("File upload aborted.")

Files = [path.abspath(File) for File in listdir()]
Files = sorted(Files, key = os.path.getmtime)

Audio = []
for File in Files:
	if File.endswith(("mp3", "flac", "ogg")):
		Audio += [File]

for File in Files:
	if File.endswith(("png", "jpg", "jpeg", "webp")):
		Cover = File
		break

#-=-=-=-#

Files = [*Audio, Cover]

print(__SepSTR)
print("\n{0}Files uploaded successfully!{1}".format(Styles.OK, Styles.Reset))

In [ ]:
try:
	chdir("/content/YTMVG")
except:
	Error("Modules not found!\nPlease run the `Setup/Setup code` cell.")

#-=-=-=-#

#@title # <font color=lime>**2.**</font> Process ⚙️
#@markdown # Thumbnail *(background color)* 🖼️
Type = "Wet" #@param ["Wet", "Dry"]
Brightness = 40 #@param {type: "slider", min: 25, max: 75, step: 0.1}
Saturation = 100 #@param {type: "slider", min: 75, max: 250, step: 0.1}

#@markdown ---

#@markdown # Video 📹
Maximum_Resolution = "1080p" #@param ["4320p", "2160p", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"]
#@markdown # Audio 🔊
Bitrate = 256 #@param {type: "slider", min: 96, max: 256, step: 1}

#@markdown ---

Debug = False #@param {type: "boolean"}

#-=-=-=-=-=-#

# CLI Variables Assignment
environ["COVER"] = Cover
environ["COLOR"] = str(1 if "t" in Type else 2)
environ["BRIGHTNESS"] = str(Brightness / 1)
environ["SATURATION"] = str(Saturation / 1)
environ["RESOLUTION"] = Maximum_Resolution.strip("p") + "p"
environ["BITRATE"] = str(Bitrate)
environ["EXEC"] = "echo " if Debug else ""
environ["DIRECTORY"] = __Output

if path.exists(__Output):
	rmtree(__Output)

#-=-=-=-#

__Counter = 0
try:
	for File in Audio:
		environ["AUDIO"] = File
		environ["NO_THUMBNAIL"] = "-nt" if __Counter else ""

		__Counter += 1
		print('{3}Processing "{0}"... [{1}/{2}]{4}'.format(
			File.split(sep)[-1], __Counter, len(Audio),
			Styles.OK, Styles.Reset
			)
		)

		#-=-=-=-#

		!$EXEC\python "YTMVG.py" \
			-a "$AUDIO" -ab $BITRATE\k \
			-i "$COVER" -r $RESOLUTION \
			-c $COLOR $NO_THUMBNAIL \
			-bgb $BRIGHTNESS\% \
			-bgs $SATURATION\% \
			-d "$DIRECTORY" -v 0 -nod

		print()

	if Debug:
		print(Styles.Reset + f"\n{__SepSTR}\n")
		!python "YTMVG.py" -h

	print("\r", end = Styles.Reset)
except KeyboardInterrupt:
	Error("YTMVG was interrupted by the user, exiting")

#-=-=-=-#
print(__SepSTR, "\n")
print("{0}Done!{1}".format(
		Styles.OK, Styles.Reset
	)
)

In [ ]:
#@title # <font color=lime>**3.**</font> Download `ZIP` with files 📥

Archive_Name = "_".join([
	path.abspath("../" + path.basename(File_Dir)),
	str(time()).split(".")[-1].zfill(7)
	]
)
Archive_Name += ".zip"

#-=-=-=-#

chdir(__Output)

# Archive Creation
print("Making Archive...")
with ZipFile(Archive_Name, "w", 14, 1, 9) as Archive:
	for File in Path().rglob("*"):
		File = str(File.resolve())
		if path.isdir(File):
			print('\t{1}Writing "{0}"...{2}'.format(
				File.split(getcwd())[-1],
				Styles.Info, Styles.Flaw
				)
			)
			Archive.write(File, File.split(sep)[-1])

files.download(Archive_Name)

#-=-=-=-#

print("\n" + __SepSTR)
print(Styles.OK + "\nDone!" + Styles.Reset)